# Workflow avec intervention humaine dans le Microsoft Agent Framework

## 🎯 Objectifs d'apprentissage

Dans ce notebook, vous apprendrez à implémenter des workflows avec **intervention humaine** en utilisant le `RequestInfoExecutor` du Microsoft Agent Framework. Ce modèle puissant permet de suspendre les workflows d'IA pour recueillir des informations auprès des humains, rendant vos agents interactifs et permettant aux humains de garder le contrôle sur les décisions critiques.

## 🔄 Qu'est-ce que l'intervention humaine ?

L'**intervention humaine (HITL)** est un modèle de conception où les agents d'IA suspendent leur exécution pour demander une entrée humaine avant de continuer. Cela est essentiel pour :

- ✅ **Décisions critiques** - Obtenez l'approbation humaine avant de prendre des actions importantes
- ✅ **Situations ambiguës** - Permettez aux humains de clarifier lorsque l'IA est incertaine
- ✅ **Préférences utilisateur** - Demandez aux utilisateurs de choisir entre plusieurs options
- ✅ **Conformité et sécurité** - Assurez une supervision humaine pour les opérations réglementées
- ✅ **Expériences interactives** - Créez des agents conversationnels qui répondent aux entrées des utilisateurs

## 🏗️ Comment cela fonctionne dans le Microsoft Agent Framework

Le framework fournit trois composants clés pour l'intervention humaine :

1. **`RequestInfoExecutor`** - Un exécuteur spécial qui suspend le workflow et émet un `RequestInfoEvent`
2. **`RequestInfoMessage`** - Classe de base pour les charges utiles typées envoyées aux humains
3. **`RequestResponse`** - Associe les réponses humaines aux demandes originales via `request_id`

**Modèle de workflow :**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Notre exemple : Réservation d'hôtel avec confirmation utilisateur

Nous allons développer le workflow conditionnel en ajoutant une confirmation humaine **avant** de suggérer des destinations alternatives :

1. L'utilisateur demande une destination (par exemple, "Paris")
2. L'`availability_agent` vérifie si des chambres sont disponibles
3. **Si aucune chambre n'est disponible** → L'`confirmation_agent` demande "Souhaitez-vous voir des alternatives ?"
4. Le workflow **se suspend** grâce au `RequestInfoExecutor`
5. **L'humain répond** "oui" ou "non" via une entrée console
6. Le `decision_manager` oriente en fonction de la réponse :
   - **Oui** → Affiche des destinations alternatives
   - **Non** → Annule la demande de réservation
7. Affiche le résultat final

Cela montre comment donner aux utilisateurs le contrôle sur les suggestions de l'agent !

---

Commençons ! 🚀


## Étape 1 : Importer les bibliothèques nécessaires

Nous importons les composants standards du cadre Agent ainsi que les **classes spécifiques à l'intervention humaine** :
- `RequestInfoExecutor` - Exécuteur qui met en pause le flux de travail pour une intervention humaine
- `RequestInfoEvent` - Événement émis lorsque l'intervention humaine est requise
- `RequestInfoMessage` - Classe de base pour les charges utiles de requêtes typées
- `RequestResponse` - Associe les réponses humaines aux requêtes
- `WorkflowOutputEvent` - Événement permettant de détecter les sorties du flux de travail


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Étape 2 : Définir des modèles Pydantic pour des sorties structurées

Ces modèles définissent le **schéma** que les agents vont retourner. Nous conservons tous les modèles du flux de travail conditionnel et ajoutons :

**Nouveau pour l'intervention humaine :**
- `HumanFeedbackRequest` - Sous-classe de `RequestInfoMessage` qui définit la charge utile de la requête envoyée aux humains
  - Contient `prompt` (question à poser) et `destination` (contexte sur la ville indisponible)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Étape 3 : Créer l'outil de réservation d'hôtel

Le même outil que celui du flux de travail conditionnel - vérifie si des chambres sont disponibles à la destination.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Étape 4 : Définir les fonctions de condition pour le routage

Nous avons besoin de **quatre fonctions de condition** pour notre flux de travail avec intervention humaine :

**Depuis le flux de travail conditionnel :**
1. `has_availability_condition` - Oriente lorsque des hôtels SONT disponibles
2. `no_availability_condition` - Oriente lorsque des hôtels NE SONT PAS disponibles

**Nouveau pour l'intervention humaine :**
3. `user_wants_alternatives_condition` - Oriente lorsque l'utilisateur répond "oui" aux alternatives
4. `user_declines_alternatives_condition` - Oriente lorsque l'utilisateur répond "non" aux alternatives


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Étape 5 : Créer l'Exécuteur du Gestionnaire de Décision

C'est **le cœur du modèle avec intervention humaine** ! Le `DecisionManager` est un `Executor` personnalisé qui :

1. **Reçoit les retours humains** via des objets `RequestResponse`
2. **Traite la décision de l'utilisateur** (oui/non)
3. **Dirige le flux de travail** en envoyant des messages aux agents appropriés

Caractéristiques principales :
- Utilise le décorateur `@handler` pour exposer des méthodes comme étapes du flux de travail
- Reçoit `RequestResponse[HumanFeedbackRequest, str]` contenant à la fois la demande originale et la réponse de l'utilisateur
- Génère des messages simples "oui" ou "non" qui déclenchent nos fonctions de condition


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Étape 6 : Créer un exécuteur d'affichage personnalisé

Même exécuteur d'affichage que celui du flux de travail conditionnel - produit les résultats finaux en tant que sortie du flux de travail.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Étape 7 : Charger les variables d'environnement

Configurez le client LLM (GitHub Models, Azure OpenAI ou OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Étape 8 : Créer des agents IA et des exécuteurs

Nous créons **six composants de workflow** :

**Agents (encapsulés dans AgentExecutor) :**
1. **availability_agent** - Vérifie la disponibilité des hôtels à l'aide de l'outil
2. **confirmation_agent** - 🆕 Prépare la demande de confirmation humaine
3. **alternative_agent** - Propose des villes alternatives (lorsque l'utilisateur dit oui)
4. **booking_agent** - Encourage la réservation (lorsque des chambres sont disponibles)
5. **cancellation_agent** - 🆕 Gère le message d'annulation (lorsque l'utilisateur dit non)

**Exécuteurs spéciaux :**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` qui met le workflow en pause pour une entrée humaine
7. **decision_manager** - 🆕 Exécuteur personnalisé qui oriente en fonction de la réponse humaine (déjà défini ci-dessus)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Étape 9 : Construire le flux de travail avec l'intervention humaine

Nous allons maintenant construire le graphe du flux de travail avec un **routage conditionnel**, incluant le chemin avec intervention humaine :

**Structure du flux de travail :**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Connexions clés :**
- `availability_agent → confirmation_agent` (lorsqu'il n'y a pas de chambres disponibles)
- `confirmation_agent → prepare_human_request` (transformation du type)
- `prepare_human_request → request_info_executor` (pause pour intervention humaine)
- `request_info_executor → decision_manager` (toujours - fournit RequestResponse)
- `decision_manager → alternative_agent` (lorsque l'utilisateur répond "oui")
- `decision_manager → cancellation_agent` (lorsque l'utilisateur répond "non")
- `availability_agent → booking_agent` (lorsque des chambres sont disponibles)
- Tous les chemins se terminent à `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Étape 10 : Exécuter le cas de test 1 - Ville SANS disponibilité (Paris avec confirmation humaine)

Ce test illustre le **cycle complet avec intervention humaine** :

1. Demander un hôtel à Paris
2. availability_agent vérifie → Pas de chambres disponibles
3. confirmation_agent crée une question destinée à l'utilisateur
4. request_info_executor **met le flux de travail en pause** et émet `RequestInfoEvent`
5. **L'application détecte l'événement et invite l'utilisateur via la console**
6. L'utilisateur tape "oui" ou "non"
7. L'application envoie la réponse via `send_responses_streaming()`
8. decision_manager décide en fonction de la réponse
9. Résultat final affiché

**Modèle clé :**
- Utiliser `workflow.run_stream()` pour la première itération
- Utiliser `workflow.send_responses_streaming(pending_responses)` pour les itérations suivantes
- Écouter `RequestInfoEvent` pour détecter quand une intervention humaine est nécessaire
- Écouter `WorkflowOutputEvent` pour capturer les résultats finaux


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Étape 11 : Exécuter le cas de test 2 - Ville AVEC disponibilité (Stockholm - Aucun input humain nécessaire)

Ce test démontre le **chemin direct** lorsque des chambres sont disponibles :

1. Demander un hôtel à Stockholm
2. availability_agent vérifie → Chambres disponibles ✅
3. booking_agent propose une réservation
4. display_result montre la confirmation
5. **Aucun input humain requis !**

Le flux contourne entièrement le chemin avec intervention humaine lorsque des chambres sont disponibles.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Points clés et bonnes pratiques pour l'humain dans la boucle

### ✅ Ce que vous avez appris :

#### 1. **Modèle RequestInfoExecutor**
Le modèle humain dans la boucle du Microsoft Agent Framework utilise trois composants clés :
- `RequestInfoExecutor` - Met en pause le flux de travail et émet des événements
- `RequestInfoMessage` - Classe de base pour les charges utiles de requêtes typées (à sous-classer !)
- `RequestResponse` - Associe les réponses humaines aux requêtes originales

**Compréhension essentielle :**
- `RequestInfoExecutor` ne collecte PAS les entrées lui-même - il se contente de mettre en pause le flux de travail
- Votre code d'application doit écouter `RequestInfoEvent` et collecter les entrées
- Vous devez appeler `send_responses_streaming()` avec un dictionnaire associant `request_id` à la réponse de l'utilisateur

#### 2. **Modèle d'exécution en streaming**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Architecture événementielle**
Écoutez des événements spécifiques pour contrôler le flux de travail :
- `RequestInfoEvent` - Une intervention humaine est nécessaire (flux de travail en pause)
- `WorkflowOutputEvent` - Le résultat final est disponible (flux de travail terminé)
- `WorkflowStatusEvent` - Changements d'état (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, etc.)

#### 4. **Exécuteurs personnalisés avec @handler**
Le `DecisionManager` montre comment créer des exécuteurs qui :
- Utilisent le décorateur `@handler` pour exposer des méthodes comme étapes du flux de travail
- Reçoivent des messages typés (par ex., `RequestResponse[HumanFeedbackRequest, str]`)
- Orientent le flux de travail en envoyant des messages à d'autres exécuteurs
- Accèdent au contexte via `WorkflowContext`

#### 5. **Routage conditionnel avec décisions humaines**
Vous pouvez créer des fonctions conditionnelles qui évaluent les réponses humaines :
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Applications concrètes :

1. **Flux de travail d'approbation**
   - Obtenir l'approbation d'un responsable avant de traiter des rapports de dépenses
   - Exiger une révision humaine avant d'envoyer des e-mails automatisés
   - Confirmer les transactions de grande valeur avant leur exécution

2. **Modération de contenu**
   - Signaler du contenu douteux pour une révision humaine
   - Demander aux modérateurs de prendre une décision finale sur les cas limites
   - Faire appel à des humains lorsque la confiance de l'IA est faible

3. **Service client**
   - Permettre à l'IA de gérer automatiquement les questions courantes
   - Escalader les problèmes complexes vers des agents humains
   - Demander au client s'il souhaite parler à un humain

4. **Traitement des données**
   - Demander aux humains de résoudre des entrées de données ambiguës
   - Confirmer les interprétations de l'IA sur des documents peu clairs
   - Permettre aux utilisateurs de choisir entre plusieurs interprétations valides

5. **Systèmes critiques pour la sécurité**
   - Exiger une confirmation humaine avant des actions irréversibles
   - Obtenir une approbation avant d'accéder à des données sensibles
   - Confirmer les décisions dans des secteurs réglementés (santé, finance)

6. **Agents interactifs**
   - Créer des bots conversationnels qui posent des questions de suivi
   - Concevoir des assistants qui guident les utilisateurs à travers des processus complexes
   - Développer des agents qui collaborent avec les humains étape par étape

### 🔄 Comparaison : Avec vs Sans humain dans la boucle

| Fonctionnalité | Flux de travail conditionnel | Flux de travail avec humain dans la boucle |
|----------------|------------------------------|-------------------------------------------|
| **Exécution** | Une seule fois avec `workflow.run()` | Boucle avec `run_stream()` + `send_responses_streaming()` |
| **Entrée utilisateur** | Aucune (entièrement automatisé) | Sollicitations interactives via `input()` ou interface utilisateur |
| **Composants** | Agents + Exécuteurs | + RequestInfoExecutor + DecisionManager |
| **Événements** | Seulement AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, etc. |
| **Pause** | Pas de pause | Le flux de travail s'arrête à RequestInfoExecutor |
| **Contrôle humain** | Aucun contrôle humain | Les humains prennent des décisions clés |
| **Cas d'utilisation** | Automatisation | Collaboration et supervision |

### 🚀 Modèles avancés :

#### Points de décision multiples pour les humains
Vous pouvez avoir plusieurs nœuds `RequestInfoExecutor` dans le même flux de travail :
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Gestion des délais
Implémentez des délais pour les réponses humaines :
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Intégration avec interface utilisateur enrichie
Au lieu de `input()`, intégrez avec une interface web, Slack, Teams, etc. :
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Humain dans la boucle conditionnel
Demandez une intervention humaine uniquement dans des situations spécifiques :
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Bonnes pratiques :

1. **Toujours sous-classer RequestInfoMessage**
   - Fournit une sécurité de type et une validation
   - Permet un contexte riche pour le rendu de l'interface utilisateur
   - Clarifie l'intention de chaque type de requête

2. **Utilisez des invites descriptives**
   - Incluez le contexte de ce que vous demandez
   - Expliquez les conséquences de chaque choix
   - Gardez les questions simples et claires

3. **Gérez les entrées inattendues**
   - Validez les réponses des utilisateurs
   - Fournissez des valeurs par défaut pour les entrées invalides
   - Donnez des messages d'erreur clairs

4. **Suivez les identifiants de requête**
   - Utilisez la corrélation entre request_id et les réponses
   - Ne tentez pas de gérer l'état manuellement

5. **Concevez pour éviter les blocages**
   - Ne bloquez pas les threads en attendant des entrées
   - Utilisez des modèles asynchrones partout
   - Prenez en charge plusieurs instances de flux de travail simultanées

### 📚 Concepts associés :

- **Middleware d'agent** - Intercepter les appels d'agent (notebook précédent)
- **Gestion de l'état du flux de travail** - Persister l'état du flux de travail entre les exécutions
- **Collaboration multi-agents** - Combiner humain dans la boucle avec des équipes d'agents
- **Architectures événementielles** - Construire des systèmes réactifs basés sur des événements

---

### 🎓 Félicitations !

Vous avez maîtrisé les flux de travail humain dans la boucle avec Microsoft Agent Framework ! Vous savez maintenant comment :
- ✅ Mettre en pause les flux de travail pour recueillir des entrées humaines
- ✅ Utiliser RequestInfoExecutor et RequestInfoMessage
- ✅ Gérer l'exécution en streaming avec des événements
- ✅ Créer des exécuteurs personnalisés avec @handler
- ✅ Orienter les flux de travail en fonction des décisions humaines
- ✅ Construire des agents IA interactifs qui collaborent avec les humains

**C'est un modèle essentiel pour construire des systèmes d'IA fiables et contrôlables !** 🚀



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction humaine professionnelle. Nous ne sommes pas responsables des malentendus ou des interprétations erronées résultant de l'utilisation de cette traduction.
